# Airline Passenger Satisfaction - Data Preparation
----
### Import data

In [ ]:
import pandas as pd

dataset_path = 'airline_passenger_satisfaction.csv'
dataset = pd.read_csv(dataset_path)
dataset.describe()

### Separate input from output

In [8]:
from sklearn.preprocessing import LabelEncoder

X = dataset.drop(['Unnamed: 0'], axis=1)            # drop ID
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(X['satisfaction'])  # output
X = X.drop(['satisfaction'], axis=1)                # remove target from input

### Handle missing values (arrival_delay)
Since the arrival_delay feature is highly correlated with the departure_delay feature, and the missing values are not that many (393 out of 129879: ~0.03%), we decide to remove the column.

In [9]:
X = X.drop(['arrival_delay_in_minutes'], axis=1)

### Encode categorical data

In [10]:
from sklearn.preprocessing import OneHotEncoder

# Get list of categorical features
s = (X.dtypes == 'object')
categorical_cols = list(s[s].index)

# Encode categorical columns (One Hot)
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(encoder.fit_transform(X[categorical_cols]))
OH_cols.index = X.index

# Replace old categorical features with their One Hot encoding
categorical = X.select_dtypes(include=['object'])
X = X.drop(categorical, axis=1)
X = pd.concat([X, OH_cols], axis=1)


### Split dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=45)

X_train.head()